In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

# Partición de datos
X = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
y = np.array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1])

# Obtener los índices de entrenamiento y prueba
train_indices, test_indices = train_test_split(np.arange(len(X)), test_size=0.3, random_state=42)

# Guardar los índices en un archivo
np.savez("indices_particion.npz", train_indices=train_indices, test_indices=test_indices)

In [2]:
train_indices

array([0, 7, 2, 9, 4, 3, 6])

In [1]:
import scipy

c:\Users\ANDRES\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
scipy.__version__

'1.7.3'

In [8]:
import boto3
from langchain_aws import ChatBedrock
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain import PromptTemplate

In [5]:
llm.invoke('hola')

AIMessage(content='¡Hola! Soy tu asistente de conversación. Estoy aquí para ayudarte con cualquier pregunta o tema que tengas en mente. ¿En qué puedo asistirte hoy? Si necesitas información, consejos, traducción, respuestas a preguntas generales o simplemente quieres charlar sobre algún tema en particular, no dudes en decírmelo. ¡Estoy aquí para ayudarte!', additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': '5b570a7e-0f5c-45cd-a371-a295099c14c6', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Mon, 16 Dec 2024 14:11:00 GMT', 'content-type': 'application/json', 'content-length': '530', 'connection': 'keep-alive', 'x-amzn-requestid': '5b570a7e-0f5c-45cd-a371-a295099c14c6'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [1053]}}, id='run-6c06d3de-c45d-48e6-a2ac-7116fd5e29f5-0', usage_metadata={'input_tokens': 2, 'output_tokens': 72, 'total_tokens': 74})

In [36]:
def clasificador_texto(texto):
    session=boto3.Session()
    bedrock=session.client(service_name='bedrock-runtime',region_name='us-east-1')
    llm=ChatBedrock(client=bedrock,model_id='anthropic.claude-3-sonnet-20240229-v1:0',beta_use_converse_api=True,region_name='us-east-1')

    response_schema=[
        ResponseSchema(name='Nombre',description='Nombre del Candidato. Por ejemplo: Raul, Juan, Oscar'),
        ResponseSchema(name='Apellido',description='Apellido del Candidato. Por ejemplo: Gomez, Salazar, Baez.'),
        ResponseSchema(name='Nombre y Apellido',description='Nombre y Apellido del Candidato. Por ejemplo: Raul Gomez'),
        ResponseSchema(name='Sexo',description='Sexo del Candidato. Las opciones son: Masculino, Femenino'),
        ResponseSchema(name='Correo',description='Correo electronico del Candidato. Responde No se menciona en caso de no encontrar la respuesta'),
        ResponseSchema(name='Telefono',description='Telefono del Candidato. Responde No se menciona en caso de no encontrar la respuesta'),
        ResponseSchema(name='Fecha de Nacimiento',description='Fecha de Nacimiento del Candidato. Responde No se menciona en caso de no encontrar la respuesta'),
        ResponseSchema(name='Formacion Academica',description='Cual es la formacion academica del candidato? Sea breve y conciso en la respuesta, citando los datos mas relevantes.'),
        ResponseSchema(name='Experiencia Laboral',description='Cual es la experiencia laboral?Sea breve y conciso en la respuesta, citando los datos mas relevantes.')
    ]

    output_parser=StructuredOutputParser.from_response_schemas(response_schema)
    format_instructions=output_parser.get_format_instructions()
    template=""" A continuacion, se te proporcionara texto sobre un curriculum vitae. En base al texto proporcionado deberas extraer la siguiente informacion con el siguiente formato:
    {format_instructions} 
    {texto}


    """

    prompt=PromptTemplate(template=template, input_variables=[texto],partial_variables={'format_instructions':format_instructions})
    #prompt=prompt.format_prompt(texto=texto).text
    #respuesta=llm.invoke(prompt)
    chain=prompt|llm|output_parser
    respuesta=chain.invoke(texto)
    return respuesta

In [39]:
clasificador_texto('me llamo andres gomez de la fuente y trabajo de repartidor')

{'Nombre': 'Andres',
 'Apellido': 'Gomez de la Fuente',
 'Nombre y Apellido': 'Andres Gomez de la Fuente',
 'Sexo': 'Masculino',
 'Correo': 'No se menciona',
 'Telefono': 'No se menciona',
 'Fecha de Nacimiento': 'No se menciona',
 'Formacion Academica': 'No se menciona',
 'Experiencia Laboral': 'Trabajo de repartidor'}

In [25]:
print(prompt)

 A continuacion, se te proporcionara texto sobre un curriculum vitae. En base al texto proporcionado deberas extraer la siguiente informacion con el siguiente formato:
    The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Nombre": string  // Nombre del Candidato. Por ejemplo: Raul, Juan, Oscar
	"Apellido": string  // Apellido del Candidato. Por ejemplo: Gomez, Salazar, Baez.
	"Nombre y Apellido": string  // Nombre y Apellido del Candidato. Por ejemplo: Raul Gomez
	"Sexo": string  // Sexo del Candidato. Las opciones son: Masculino, Femenino
	"Correo": string  // Correo electronico del Candidato. Responde No se menciona en caso de no encontrar la respuesta
	"Telefono": string  // Telefono del Candidato. Responde No se menciona en caso de no encontrar la respuesta
	"Fecha de Nacimiento": string  // Fecha de Nacimiento del Candidato. Responde No se menciona en caso de no encontrar la respuest

In [14]:
prompt=clasificador_texto('hola')

In [ ]:
prompt=prompt.format_prompt(texto=texto).text

In [76]:
import re
import ast
def fortalezas_y_debilidades(texto,descripcion_trabajo):
    session=boto3.Session()
    bedrock=session.client(service_name='bedrock-runtime',region_name='us-east-1')
    llm=ChatBedrock(client=bedrock,model_id='anthropic.claude-3-sonnet-20240229-v1:0',region_name='us-east-1')

    response_schema=[
        ResponseSchema(name='Fortalezas',description='Cuales son las Fortalezas del Candidato con respecto al la descripcion del trabajo?'),
        ResponseSchema(name='Debilidades',description='Cuales son las Debilidades del Candidato con respecto al la descripcion del trabajo?'),
        ResponseSchema(name='Categoria',description='En que Area de la organizacion puede el candidato encajar basado en sus aptitudes y habilidades?'),
    ]

    output_parser=StructuredOutputParser.from_response_schemas(response_schema)
    format_instructions=output_parser.get_format_instructions()
    template=""" 
    Actua como un reclutador de talentos con experiencia en el ambito bancario. 
    En base a la descripcion del puesto de trabajo que se te proporcionara deberas realizar un analisis 
    con las siguientes instrucciones tomando en cuenta los datos del curriculum vitae. 
    Tu respuesta debe ser bien detallada y precisa.
   A continuacion se presentan las instrucciones:
    {format_instructions} \n
    La descripcion de la busqueda del trabajo son las siguientes:
   {descripcion_trabajo}
    A continuacion se presentan los datos del curriculum vitae de la persona:
    {texto}

    """

    prompt = PromptTemplate(
    template=template,
    input_variables=["descripcion_trabajo", "texto"],
    partial_variables={"format_instructions": format_instructions})
    #prompt=prompt.format_prompt(texto=texto).text
    #respuesta=llm.invoke(prompt)
    prompt_text = prompt.format(descripcion_trabajo=descripcion_trabajo, texto=texto)
    respuesta=llm.invoke(prompt_text).content
    pattern=r'({[^}]+})'
    matches=re.findall(pattern,respuesta)
    diccionario=ast.literal_eval(matches[0])
    fortalezas=diccionario['Fortalezas']
    debilidades=diccionario['Debilidades']
    categoria=diccionario['Categoria']
    return fortalezas,debilidades,categoria

In [77]:
descripcion_trabajo = "Analista de riesgos financieros con experiencia en banca. Necesita conocimiento en Python "
texto = "Juan Pérez, economista con 5 años de experiencia en análisis financiero y manejo de riesgos."

In [78]:
fortalezas,debilidades,categoria=fortalezas_y_debilidades(texto,descripcion_trabajo)

In [79]:
categoria

'Gestión de Riesgos Financieros'

In [62]:
debilidades

'Si bien Juan Pérez tiene experiencia en el ámbito financiero y de riesgos, el currículum no menciona específicamente conocimientos en Python, que es un requerimiento clave para el puesto de Analista de Riesgos Financieros. Esta falta de mención sobre habilidades en programación podría ser una debilidad a considerar.'

In [67]:
requerimientos='Debe jugar futbol profesionalmente'

In [70]:
def requerimientos_excluyentes(texto,requerimientos):
    session=boto3.Session()
    bedrock=session.client(service_name='bedrock-runtime',region_name='us-east-1')
    llm=ChatBedrock(client=bedrock,model_id='anthropic.claude-3-sonnet-20240229-v1:0',region_name='us-east-1')
    template='''
    Actua como un reclutador de talentos con experiencia en el ambito bancario.
    A continuacion, se te proporcionara informacion sobre el curriculum vitae de un candidato 
    y tambien los requerimientos excluyentes de la busqueda de la persona que se desea contratar. 
    Deberas responder solamente con:
    Excluyente: en caso de que no cumpla con los requerimientos.
    No Excluyente: en caso de que cumpla con los requerimientos.
    Las opciones de respuesta son: Excluyente, No Excluyente
    No debes agregar texto extra a la respuesta, debes ser directo y conciso con la respuesta
    Los Requerimientos son los siguientes: \n 
    {requerimientos}
    La informacion del Curriculum Vitae del candidato es la siguiente: \n
    {texto}
    '''

    prompt = PromptTemplate(
    template=template,
    input_variables=["texto", "requerimientos"])
    prompt_text = prompt.format(requerimientos=requerimientos, texto=texto)
    respuesta=llm.invoke(prompt_text).content
    return respuesta

In [71]:
requerimientos_excluyentes(texto, requerimientos)

'Excluyente'